In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
### Hyperparameters babyyy, change accordingly
Channel_uses = 4
Epochs = 10000
Noise_variance = 1e-4
Pert_variance = 1e-4
Batch_size = 1024

In [3]:
def tx_loss(y_true, y_pred): 
    # For numerical stability, we skipped the log and exp in the cost
    # function and  policy function. 
    return -y_true*y_pred

In [4]:
def perturbation(d):
    W = tf.keras.backend.random_normal(shape = (2*Channel_uses,),
    mean=0.0,stddev=Pert_variance**0.5,dtype=None,seed=None)
    d = ((1-Pert_variance)**0.5)*d + W
    return d

In [5]:
def batch_normalise_ret(data):
    mean = np.mean(data)
    std = np.std(data)
    normalised = (data-mean)/(std)
    return mean, std, normalised

def denormalise(data, mean, std):
    data = (std*data) + mean
    return data

###  Tx model

In [6]:
def blah(y):
    w = y[:,y.shape[-1]//2:] - y[:,:y.shape[-1]//2]
    print('hehehe', w.shape)
    t = -keras.backend.sum(w*w)/Pert_variance**2 + np.log((np.pi*Pert_variance**2)**Channel_uses)
    return t

In [7]:
# tx layers
tx_input = keras.layers.Input((1,), name='tx_input')
# x = keras.layers.BatchNormalization()(tx_input)
# Removing the batch normalisation layer and substituting it with a global function to maintain 
# uniformity while shrinking and inflating.
x = tx_input
x = keras.layers.Dense(units=10*Channel_uses, activation='elu', name='tx_10')(x)
x = keras.layers.Dense(units=2*Channel_uses, activation='elu', name='tx_out')(x)
xp = keras.layers.Lambda(perturbation, name='Xp')(x)
concat = keras.layers.concatenate([x,xp], axis=1)
policy = keras.layers.Lambda(blah)(concat)
print(concat.shape)

hehehe (?, 8)
(?, 16)


In [8]:
tx_model = keras.models.Model(inputs=tx_input, outputs=concat)

In [9]:
tx_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tx_input (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
tx_10 (Dense)                   (None, 40)           80          tx_input[0][0]                   
__________________________________________________________________________________________________
tx_out (Dense)                  (None, 8)            328         tx_10[0][0]                      
__________________________________________________________________________________________________
Xp (Lambda)                     (None, 8)            0           tx_out[0][0]                     
__________________________________________________________________________________________________
concatenat

In [10]:
pert_model = keras.models.Model(inputs=tx_input, outputs=policy)

In [11]:
pert_model.compile(loss=tx_loss, optimizer='sgd')
pert_model.summary()

TypeError: 'NoneType' object cannot be interpreted as an integer

### Rx model

In [ ]:
rx_input = keras.layers.Input((2*Channel_uses,), name='rx_input')
# channel layer
y = keras.layers.Lambda(lambda x: x+keras.backend.random_normal(
        shape = (2*Channel_uses,), mean=0.0, stddev=Noise_variance**0.5), name='channel')(rx_input)

y = keras.layers.Dense(2*Channel_uses, activation='relu', name='rx_2')(y)
y = keras.layers.Dense(10*Channel_uses, activation='relu', name='rx_10')(y)
pred = keras.layers.Dense(1, activation='relu', name='rx_output')(y)

In [ ]:
rx_model = keras.models.Model(inputs=rx_input, outputs=pred)
rx_model.summary()

In [ ]:
rx_model.compile(loss='mse', optimizer='sgd')

In [ ]:
### Data generator for consistency sake
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([0, 1]))

In [ ]:
data_numbers = np.random.uniform(0,1,(Batch_size,))
y = tx_model.predict(data_numbers)
print(y.shape)
XP = y[:,y.shape[-1]//2:]
estimated_vector = np.squeeze(rx_model.predict(XP))
print(estimated_vector.shape, data_numbers.shape)

In [ ]:
l = (estimated_vector-data_numbers)**2
l_hat = rx_model.predict(tx_model.predict(data_numbers)[:,2*Channel_uses:])

In [ ]:
pert_model.fit(data_numbers, l_hat, batch_size=Batch_size, epochs=1)

### Training

In [ ]:
for i in range(Epochs):
    data_numbers = np.random.uniform(0,1,(Batch_size,))
    y = tx_model.predict(data_numbers)
    XP = y[:,y.shape[-1]//2:]
    estimated_vector= np.squeeze(rx_model.predict(XP))
    l = (estimated_vector-data_numbers)**2
    l_hat = rx_model.predict(tx_model.predict(data_numbers)[:,2*Channel_uses:])
    pert_model.fit(data_numbers, l_hat, batch_size=Batch_size, epochs=1, verbose=0)
    rx_model.fit(XP, data_numbers, batch_size=Batch_size, epochs=1)

In [ ]:
data_numbers = np.random.uniform(0,1,(10,))
y = tx_model.predict(data_numbers)
XP = y[:,y.shape[-1]//2:]
estimated_vector= np.squeeze(rx_model.predict(XP))
print(data_numbers)
print(estimated_vector)
# l = (estimated_vector-data_numbers)**2
# l_hat = rx_model.predict(tx_model.predict(data_numbers)[:,2*Channel_uses:])
# pert_model.fit(data_numbers, l_hat, batch_size=Batch_size, epochs=1)
# print("Tx-done")
# rx_model.fit(XP, data_numbers, batch_size=Batch_size, epochs=1)
# print("Rx-done")